In [1]:
import speech_recognition as sr
import pyttsx3

In [2]:
str = "002.Laysan_Albatross 012.Yellow_headed_Blackbird 014.Indigo_Bunting 025.Pelagic_Cormorant 029.American_Crow 033.Yellow_billed_Cuckoo 035.Purple_Finch 042.Vermilion_Flycatcher 048.European_Goldfinch 050.Eared_Grebe 059.California_Gull 068.Ruby_throated_Hummingbird 073.Blue_Jay 081.Pied_Kingfisher 095.Baltimore_Oriole 101.White_Pelican 106.Horned_Puffin 108.White_necked_Raven 112.Great_Grey_Shrike 118.House_Sparrow 134.Cape_Glossy_Starling 138.Tree_Swallow 144.Common_Tern 191.Red_headed_Woodpecker"
l = str.split(" ")
l = [i.split(".")[1] for i in l]
l = [i.replace("_", " ").lower() for i in l]

In [3]:
list_of_birds = l

In [4]:
list_of_birds

['laysan albatross',
 'yellow headed blackbird',
 'indigo bunting',
 'pelagic cormorant',
 'american crow',
 'yellow billed cuckoo',
 'purple finch',
 'vermilion flycatcher',
 'european goldfinch',
 'eared grebe',
 'california gull',
 'ruby throated hummingbird',
 'blue jay',
 'pied kingfisher',
 'baltimore oriole',
 'white pelican',
 'horned puffin',
 'white necked raven',
 'great grey shrike',
 'house sparrow',
 'cape glossy starling',
 'tree swallow',
 'common tern',
 'red headed woodpecker']

In [5]:
recognizer = sr.Recognizer()

while True:
    try:
        with sr.Microphone() as source:
            print("Listening...")
            recognizer.adjust_for_ambient_noise(source, duration=0.2)
            audio = recognizer.listen(source)
            print("Processing...")
            text = recognizer.recognize_google(audio, language="en-US")
            text = text.lower()
            print("You said:", text)
    except sr.UnknownValueError:
        print("Sorry, I did not understand that.")
        continue

Listening...


KeyboardInterrupt: 

In [6]:
try:
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        audio = recognizer.listen(source)
        print("Processing...")
        text = recognizer.recognize_google(audio, language="en-US")
        text = text.lower()
        print("You said:", text)
except sr.UnknownValueError:
    print("Sorry, I did not understand that.")

Listening...
Processing...
You said: hello


In [9]:
engine = pyttsx3.init()
engine.setProperty('rate', 130)         # words per minute
engine.setProperty('volume', 1)       # 0.0–1.0
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[111].id)  # pick a voice - I pick Sandy

engine.say("Hello! Your bird catalogue is ready.")
engine.runAndWait()

2025-05-19 08:16:28.411 python[17252:303161804] Could not retrieve voice [AVSpeechSynthesisProviderVoice 0x120161250] Name: Sandy, Identifier: com.apple.eloquence.en-US.Sandy, Supported Languages (
    "en-US"
), Age: 0, Gender: 0, Size: 0, Version: (null)
2025-05-19 08:16:28.415 python[17252:303161804] Could not retrieve voice [AVSpeechSynthesisProviderVoice 0x120161250] Name: Sandy, Identifier: com.apple.eloquence.en-US.Sandy, Supported Languages (
    "en-US"
), Age: 0, Gender: 0, Size: 0, Version: (null)
2025-05-19 08:16:28.416 python[17252:303161804] Could not retrieve voice [AVSpeechSynthesisProviderVoice 0x120161250] Name: Sandy, Identifier: com.apple.eloquence.en-US.Sandy, Supported Languages (
    "en-US"
), Age: 0, Gender: 0, Size: 0, Version: (null)
2025-05-19 08:16:28.417 python[17252:303161804] Could not retrieve voice [AVSpeechSynthesisProviderVoice 0x120161250] Name: Sandy, Identifier: com.apple.eloquence.en-US.Sandy, Supported Languages (
    "en-US"
), Age: 0, Gender: 

In [8]:
# voices = engine.getProperty('voices')
# for i, v in enumerate(voices):
#     if 'en_US' in v.languages:
#         print(f"{i}: id={v.id!r}")
#         print(f"   name        : {v.name}")
#         print(f"   languages   : {v.languages}")
#         print(f"   gender      : {v.gender}")
#         print(f"   age         : {v.age}\n")

In [10]:
engine.say("eared grebe")
engine.runAndWait()

In [11]:
engine.say("Hello dear lady, which is your favorite bird?")
engine.runAndWait()


In [16]:
from rapidfuzz import process, fuzz
# def snap_bird(word):
#     match, score, _ = process.extractOne(word, list_of_birds, scorer=fuzz.WRatio)
#     return match if score > 80 else word


In [13]:
final_words = [snap_bird(w) for w in text.split()]

In [14]:
def snap_birds(text, birds, threshold=85):
    """
    Find substrings in `text` that closely match any bird name in `birds`
    and replace them with the canonical spelling.
    """
    words = text.split()            # coarse tokenisation
    fixed = []
    i = 0
    best_best = ""
    best_score = 0
    while i < len(words):
        # try to match up to 3‑word windows (adjust as needed)
        matched = False
        for window in range(3, 0, -1):
            chunk = " ".join(words[i:i+window])
            if not chunk:
                continue
            best, score, _ = process.extractOne(
                chunk,
                birds,
                scorer=fuzz.WRatio
            )
            if score >= threshold and score > best_score:
                fixed.append(best)  # canonical bird name
                i += window
                matched = True
                best_best = best
                best_score = score
                break
        if not matched:
            fixed.append(words[i])
            i += 1
    return best_best
    # return " ".join(fixed)
    
text = ""

try:
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        audio = recognizer.listen(source)
        print("Processing...")
        text = recognizer.recognize_google(audio, language="en-US")
        text = text.lower()
        # print("You said:", text)
except sr.UnknownValueError:
    print("Sorry, I did not understand that.")

clean_text = snap_birds(text, list_of_birds, threshold=90)
print("You said: ", text)
print("Bird species: ", clean_text)


Listening...
Processing...
Sorry, I did not understand that.
You said:  
Bird species:  


In [15]:
print(list_of_birds)

['laysan albatross', 'yellow headed blackbird', 'indigo bunting', 'pelagic cormorant', 'american crow', 'yellow billed cuckoo', 'purple finch', 'vermilion flycatcher', 'european goldfinch', 'eared grebe', 'california gull', 'ruby throated hummingbird', 'blue jay', 'pied kingfisher', 'baltimore oriole', 'white pelican', 'horned puffin', 'white necked raven', 'great grey shrike', 'house sparrow', 'cape glossy starling', 'tree swallow', 'common tern', 'red headed woodpecker']


In [17]:
import json, queue, sys, time
import sounddevice as sd
from vosk import Model, KaldiRecognizer

MODEL_PATH = "models/vosk-model-small-en-us-0.15"
SAMPLE_RATE = 16000  # model sample rate

# 1. Load model
model = Model(MODEL_PATH)

# 2. Build phrase-bias JSON (not a hard grammar)
grammar_json = json.dumps({"phrase_list": list_of_birds})
rec = KaldiRecognizer(model, SAMPLE_RATE, grammar_json)

# # 3. rapidfuzz helper
# def snap_birds(text, birds=BIRDS, threshold=85):
#     words = text.split()
#     fixed = []
#     i = 0
#     while i < len(words):
#         matched = False
#         for window in range(4, 0, -1):          # up to 4-word chunks
#             chunk = " ".join(words[i:i+window])
#             if not chunk:
#                 continue
#             best, score, _ = process.extractOne(
#                 chunk, birds, scorer=fuzz.WRatio
#             )
#             if score >= threshold:
#                 fixed.append(best)              # canonical name
#                 i += window
#                 matched = True
#                 break
#         if not matched:
#             fixed.append(words[i])
#             i += 1
#     return " ".join(fixed)

# 4. Audio callback
q = queue.Queue()
def callback(indata, frames, time_, status):
    if status:
        print(status, file=sys.stderr)
    q.put(bytes(indata))

# 5. Start stream + recognizer loop
print("🎤  Speak; Ctrl-C to stop.")
with sd.RawInputStream(samplerate=SAMPLE_RATE,
                       blocksize=8000,
                       dtype="int16",
                       channels=1,
                       callback=callback):

    partial_printed = ""
    try:
        while True:
            data = q.get()
            if rec.AcceptWaveform(data):
                result = json.loads(rec.Result())
                text = result.get("text", "")
                if text:
                    fixed = snap_birds(text.lower())
                    print(f"\n✅  {fixed}")
                    partial_printed = ""
            else:
                # show partials in same line
                partial = json.loads(rec.PartialResult()).get("partial", "")
                if partial and partial != partial_printed:
                    sys.stdout.write("\r… " + partial + "   ")
                    sys.stdout.flush()
                    partial_printed = partial
    except KeyboardInterrupt:
        print("\nStopped.")


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from models/vosk-model-small-en-us-0.15/graph/HCLr.fst models/vosk-model-small-en-us-0.15/graph/Gr.fst


: 

: 

In [1]:
# import json, queue
# import sounddevice as sd
import keyboard
# from vosk import Model, KaldiRecognizer

# BIRDS = ["blue jay", "great tit", "american crow"]   # …
# model = Model("models/vosk-model-small-en-us-0.15")
# rec   = KaldiRecognizer(model, 16000, json.dumps(BIRDS))

# q = queue.Queue()
# def audio_cb(indata, frames, time, status):
#     q.put(bytes(indata))

# def listen_once():
#     with sd.RawInputStream(samplerate=16000, blocksize=8000,
#                            dtype="int16", channels=1,
#                            callback=audio_cb):
#         print("⏺️  Listening… release key to stop")
#         while keyboard.is_pressed('space'):
#             try:
#                 data = q.get(timeout=0.1)
#                 if rec.AcceptWaveform(data):
#                     print(json.loads(rec.Result())['text'])
#             except queue.Empty:
#                 pass
#         print("⏹️  Paused")

print("Hold SPACE to talk.  Press ESC to quit.")
# while True:
#     keyboard.wait('space')
#     print("listening...")
#     if keyboard.is_pressed('esc'):
#         break

: 

: 